# import 

In [13]:
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset
import time
from glob import glob
import re
from matplotlib import pyplot as plt
%matplotlib inline

os.environ["CUDA_VISIBLE_DEVICES"]='2' 

from torchsummary import summary


# data

In [2]:
def get_paths_and_labels(img_type='training', isLbael=True):
    base_dir = '/workdir/home/feynman52/NTU-ML2020/hw3-Food-Classification-by-CNN/datasets'
    paths = sorted(glob(os.path.join(base_dir, img_type, '*')))[:]
    
    if isLbael==True: 
        Y = [int(re.search('/(.{1,2})_', path).group(1)) for path in paths]
        return paths, Y
    else:
        return paths

In [3]:
x_train_paths, y_train = get_paths_and_labels(img_type='training', isLbael=True)
x_valid_paths, y_valid = get_paths_and_labels(img_type='validation', isLbael=True)
x_test_paths = get_paths_and_labels(img_type='testing', isLbael=False)

len(x_train_paths), len(y_train), len(x_valid_paths), len(y_valid), len(x_test_paths)


(9866, 9866, 3430, 3430, 3347)

In [4]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(15), 
    transforms.ToTensor(), 
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
])

In [5]:
class ImgDataset(Dataset):
    def __init__(self, img_paths, labels=None, transform=None):
        self.img_paths = img_paths
        
        self.labels = labels
        if self.labels != None:
            self.labels = torch.LongTensor(labels) ###
            
        self.transform = transform
        
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))
        
        if self.transform!=None: img = self.transform(img)
            
        if self.labels==None: 
            return img
        else:
            label = self.labels[index]
            return img, label
            

In [6]:
train_set = ImgDataset(x_train_paths, y_train, train_transform)
valid_set = ImgDataset(x_valid_paths, y_valid, test_transform)

In [7]:
x, y = train_set[-1]
x.shape, y

(torch.Size([3, 128, 128]), tensor(9))

In [8]:
batch_size = 50
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True) # shuffle select index
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)


In [9]:
g = iter(train_loader)
x_batch, y_batch = next(g)
x_batch.shape, y_batch.shape
x_batch.size(), y_batch.size()

(torch.Size([50, 3, 128, 128]), torch.Size([50]))

In [10]:
len(train_loader), len(valid_loader)

(198, 69)

# model

## original model

In [11]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, in_):
        x = self.cnn(in_)
        #print(x.shape)
        x = x.reshape(x.shape[0], -1) # x.shape = x.sise()
        out_ = self.fc(x)
        return out_

In [15]:
model = Classifier().cuda()
summary(model, input_size=(3, 128, 128))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 128, 128]           1,792
       BatchNorm2d-2         [-1, 64, 128, 128]             128
              ReLU-3         [-1, 64, 128, 128]               0
         MaxPool2d-4           [-1, 64, 64, 64]               0
            Conv2d-5          [-1, 128, 64, 64]          73,856
       BatchNorm2d-6          [-1, 128, 64, 64]             256
              ReLU-7          [-1, 128, 64, 64]               0
         MaxPool2d-8          [-1, 128, 32, 32]               0
            Conv2d-9          [-1, 256, 32, 32]         295,168
      BatchNorm2d-10          [-1, 256, 32, 32]             512
             ReLU-11          [-1, 256, 32, 32]               0
        MaxPool2d-12          [-1, 256, 16, 16]               0
           Conv2d-13          [-1, 512, 16, 16]       1,180,160
      BatchNorm2d-14          [-1, 512,

## prune model

In [1]:
1792/280

6.4

In [21]:
class ClassifierPrune(nn.Module):
    def __init__(self):
        super(ClassifierPrune, self).__init__()
        
        self.Conv2d_prune = nn.Sequential(
        )
        
        self.cnn = nn.Sequential(
            #nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            self.make_prune_cnn(3, 64, 3, 1, 1),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            #nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            self.make_prune_cnn(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            #nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            self.make_prune_cnn(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]

            #nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            self.make_prune_cnn(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            #nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            self.make_prune_cnn(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def make_prune_cnn(self, in_chs, out_chs, kernel_size, stride, padding):
        prune_cnn = nn.Sequential(
            nn.Conv2d(in_chs, in_chs, kernel_size, stride, padding, groups=in_chs), # depthwise
            nn.Conv2d(in_chs, out_chs, 1) # pointwise
        )
        return prune_cnn
        
    def forward(self, in_):
        x = self.cnn(in_)
        x = x.reshape(x.shape[0], -1) 
        out_ = self.fc(x)
        return out_

In [24]:
(1792)/(30+256)

6.265734265734266

In [22]:
model_prune = ClassifierPrune().cuda()
summary(model_prune, input_size=(3, 128, 128))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 3, 128, 128]              30
            Conv2d-2         [-1, 64, 128, 128]             256
       BatchNorm2d-3         [-1, 64, 128, 128]             128
              ReLU-4         [-1, 64, 128, 128]               0
         MaxPool2d-5           [-1, 64, 64, 64]               0
            Conv2d-6           [-1, 64, 64, 64]             640
            Conv2d-7          [-1, 128, 64, 64]           8,320
       BatchNorm2d-8          [-1, 128, 64, 64]             256
              ReLU-9          [-1, 128, 64, 64]               0
        MaxPool2d-10          [-1, 128, 32, 32]               0
           Conv2d-11          [-1, 128, 32, 32]           1,280
           Conv2d-12          [-1, 256, 32, 32]          33,024
      BatchNorm2d-13          [-1, 256, 32, 32]             512
             ReLU-14          [-1, 256,

# train

## original model

In [12]:
print('train')

model = Classifier().cuda()
epochs = 10

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

for epoch in range(epochs):
    # initialize time
    epoch_start_time = time.time()
    # initialize metric
    train_acc = 0.
    valid_acc = 0.
    train_loss = 0.
    valid_loss = 0.
    
    # -----------------------
    #  train 
    # -----------------------
    model.train() ###
    for (i, data) in enumerate(train_loader):
        x, y = data[0].cuda(), data[1].cuda()
        
        # reset gradient !!!
        optimizer.zero_grad()
        
        # y_hat, (-1,11)
        y_hat = model(x)
        
        # loss
        batch_loss = loss(y_hat, y)
        
        # gradient
        batch_loss.backward()
        
        # update_weight
        optimizer.step()
        
        # metric, acc, loss
        label_hat = np.argmax(y_hat.cpu().data.numpy(), axis=1)
        label = y.cpu().data.numpy()
        match = (label_hat==label)
        train_acc += sum(match)
        train_loss += batch_loss.item()
        

    
    # -----------------------
    #  valid 
    # -----------------------
    model.eval() ###
    with torch.no_grad(): ###
        for (i, data) in enumerate(valid_loader):
            x, y = data[0].cuda(), data[1].cuda()

            # y_hat
            y_hat = model(x)

            # loss
            batch_loss = loss(y_hat, y)

            # metric
            label_hat = np.argmax(y_hat.cpu().data.numpy(), axis=1)
            label = y.cpu().data.numpy()
            match = (label_hat==label)
            valid_acc += sum(match)

            valid_loss += batch_loss.item()
    
    
    # -----------------------
    #  progress 
    # -----------------------
    epoch += 1
    t = time.time() - epoch_start_time
    train_loss /= len(train_set)
    valid_loss /= len(valid_set)
    train_acc /= len(train_set)
    valid_acc /= len(valid_set)

    print('epoch = %d, time = %d, train_loss = %.3f, train_acc = %.2f, valid_loss = %.3f, valid_acc = %.2f'%(
        epoch, t, train_loss, train_acc, valid_loss, valid_acc))


train
epoch = 1, time = 81, train_loss = 0.042, train_acc = 0.28, valid_loss = 0.041, valid_acc = 0.30
epoch = 2, time = 81, train_loss = 0.036, train_acc = 0.38, valid_loss = 0.037, valid_acc = 0.35
epoch = 3, time = 81, train_loss = 0.033, train_acc = 0.43, valid_loss = 0.034, valid_acc = 0.40
epoch = 4, time = 80, train_loss = 0.031, train_acc = 0.46, valid_loss = 0.034, valid_acc = 0.41
epoch = 5, time = 80, train_loss = 0.028, train_acc = 0.50, valid_loss = 0.032, valid_acc = 0.48
epoch = 6, time = 80, train_loss = 0.027, train_acc = 0.53, valid_loss = 0.028, valid_acc = 0.50
epoch = 7, time = 81, train_loss = 0.025, train_acc = 0.56, valid_loss = 0.032, valid_acc = 0.52
epoch = 8, time = 80, train_loss = 0.023, train_acc = 0.59, valid_loss = 0.034, valid_acc = 0.48
epoch = 9, time = 80, train_loss = 0.022, train_acc = 0.61, valid_loss = 0.032, valid_acc = 0.50
epoch = 10, time = 80, train_loss = 0.021, train_acc = 0.64, valid_loss = 0.025, valid_acc = 0.59


## prune model

In [23]:
print('train')

model_prune_prune = ClassifierPrune().cuda()
epochs = 10

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model_prune.parameters(), lr=0.001)

for epoch in range(epochs):
    # initialize time
    epoch_start_time = time.time()
    # initialize metric
    train_acc = 0.
    valid_acc = 0.
    train_loss = 0.
    valid_loss = 0.
    
    # -----------------------
    #  train 
    # -----------------------
    model_prune.train() ###
    for (i, data) in enumerate(train_loader):
        x, y = data[0].cuda(), data[1].cuda()
        
        # reset gradient !!!
        optimizer.zero_grad()
        
        # y_hat, (-1,11)
        y_hat = model_prune(x)
        
        # loss
        batch_loss = loss(y_hat, y)
        
        # gradient
        batch_loss.backward()
        
        # update_weight
        optimizer.step()
        
        # metric, acc, loss
        label_hat = np.argmax(y_hat.cpu().data.numpy(), axis=1)
        label = y.cpu().data.numpy()
        match = (label_hat==label)
        train_acc += sum(match)
        train_loss += batch_loss.item()
        

    
    # -----------------------
    #  valid 
    # -----------------------
    model_prune.eval() ###
    with torch.no_grad(): ###
        for (i, data) in enumerate(valid_loader):
            x, y = data[0].cuda(), data[1].cuda()

            # y_hat
            y_hat = model_prune(x)

            # loss
            batch_loss = loss(y_hat, y)

            # metric
            label_hat = np.argmax(y_hat.cpu().data.numpy(), axis=1)
            label = y.cpu().data.numpy()
            match = (label_hat==label)
            valid_acc += sum(match)

            valid_loss += batch_loss.item()
    
    
    # -----------------------
    #  progress 
    # -----------------------
    epoch += 1
    t = time.time() - epoch_start_time
    train_loss /= len(train_set)
    valid_loss /= len(valid_set)
    train_acc /= len(train_set)
    valid_acc /= len(valid_set)

    print('epoch = %d, time = %d, train_loss = %.3f, train_acc = %.2f, valid_loss = %.3f, valid_acc = %.2f'%(
        epoch, t, train_loss, train_acc, valid_loss, valid_acc))


train
epoch = 1, time = 78, train_loss = 0.042, train_acc = 0.29, valid_loss = 0.044, valid_acc = 0.25
epoch = 2, time = 78, train_loss = 0.034, train_acc = 0.41, valid_loss = 0.032, valid_acc = 0.43
epoch = 3, time = 78, train_loss = 0.029, train_acc = 0.49, valid_loss = 0.036, valid_acc = 0.43
epoch = 4, time = 78, train_loss = 0.026, train_acc = 0.55, valid_loss = 0.027, valid_acc = 0.54
epoch = 5, time = 78, train_loss = 0.024, train_acc = 0.59, valid_loss = 0.028, valid_acc = 0.53
epoch = 6, time = 78, train_loss = 0.022, train_acc = 0.62, valid_loss = 0.029, valid_acc = 0.52
epoch = 7, time = 78, train_loss = 0.021, train_acc = 0.64, valid_loss = 0.027, valid_acc = 0.56
epoch = 8, time = 78, train_loss = 0.019, train_acc = 0.67, valid_loss = 0.025, valid_acc = 0.60
epoch = 9, time = 78, train_loss = 0.018, train_acc = 0.69, valid_loss = 0.023, valid_acc = 0.63
epoch = 10, time = 78, train_loss = 0.017, train_acc = 0.70, valid_loss = 0.022, valid_acc = 0.64
